In [25]:
import pandas as pd
import re
import string
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import csv
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Siddharth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
removeIndex = []

In [3]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [4]:
def createlabel(df):
    labels = []
    for i in range(len(df)):
        try:
            labels.append(int(list(df.loc[i].dropna())[-1]))
        except:
            removeIndex.append(i)
    return labels

In [5]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [6]:


def formatname(line):
    try:
        s = line
        res = re.sub('[%s]' % re.escape(string.punctuation), '' , s)
        res = re.sub(r'\w*\d\w*','',res.lower())
        res = lemmatize_words(res)
        return res
    except:
        return 0

In [7]:


def findDescription(arr):
    #print(arr)
    stri = []
    arr = list(arr)
    for i in range(1, len(arr)):
        
        if(str(arr[i])[0] == '['):
            break
        else:
            stri.append(str(arr[i]))
    return ''.join(dict.fromkeys(stri))
def createDescription(line):
    try:
        s = line
        res = cleanhtml(s)
        res = re.sub('[%s]' % re.escape(string.punctuation), '' , s)
        res = re.sub(r'\w*\d\w*','',res.lower())
        res = lemmatize_words(res)
        return res
    except:
        return 0
        


In [8]:


def extractBulletPoints(a):
    a = list(a)
    start = 0
    end = 0
    for i in range(len(a)):
        if(str(a[i])[0] == '['):
            start = i
            break
    for i in range(len(a)):
        if(str(a[i])[-1] == ']'):
            end = i
            break
    if(start != end):
        return a[start: end+1]
    return []
def createBulletPoints(df):
    bulletpoints = []
    
    for i in range(0, len(df)):
        s = df.loc[i].dropna()
        res = extractBulletPoints(s)
        if(len(res)):
            try:
                res = ''.join(dict.fromkeys(res))
                res = re.sub('[%s]' % re.escape(string.punctuation), '' , res)
                res = re.sub(r'\w*\d\w*','',res.lower())
                res = lemmatize_words(res)
                res = ' '.join(dict.fromkeys(res.split()))
                bulletpoints.append(res)
            except:
                bulletpoints.append('-1')
        else:
            bulletpoints.append('-1')
    return bulletpoints
  


    


In [9]:
def createBrand(line):
    try:
        name = line.split()
        if(len(name) <= 6):
            return ''.join(name)
        else:
            return 0
    except:
        return 0

In [10]:
def loadFile(name):
    df = pd.read_csv('dataset/%s' % name,header = None)
    return df

In [11]:
def dropAnamoli(df):
    
    for i in range(len(df)):
        if(len(df.loc[i].dropna()) < 5):
            df = df.drop([i])
    df = df.reset_index(drop=True)

In [12]:
def createCsv(df,j):
    
    fields = ['PRODUCT_ID', 'TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND','BROWSE_NODE_ID']
    rows = []
    for i in range(len(df)):
        rows.append([i, name[i], description[i], bulletpoint[i], brand[i], label[i]])
    with open('tune%s.csv' % str(int(j)+9), 'w',encoding="utf-8") as csvfile: 
    # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
        csvwriter.writerow(fields) 
        
    # writing the data rows 
        csvwriter.writerows(rows)
    


In [ ]:
files = ['train1.csv','train10001.csv','train15001.csv','train20001.csv','train25001.csv','train30001.csv','train35001.csv','train40001.csv','train45001.csv','train50001.csv','train5001.csv']
for i in range(len(files)):
    df = loadFile(files[i])
    dropAnamoli(df)
    description = createDescription(df)
    label  = createlabel(df)
    name = createname(df)
    brand = createBrand(df)
    bulletpoint = createBulletPoints(df)
    a = list(set(removeIndex))
    print(a)
    for j in a:
        df = df.drop([j])
    df = df.reset_index(drop = True)
    label  = createlabel(df)
    name = createname(df)
    brand = createBrand(df)
    bulletpoint = createBulletPoints(df)
    description = createDescription(df)
    createCsv(df,i)
    
    
    
    

In [ ]:

df_from_each_file = (pd.read_csv('tune%s.csv' % str(f), sep=',') for f in range(9,20))
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "tune.csv")

In [ ]:
df = pd.read_csv('tune.csv')
df


In [ ]:
df.fillna(-1)

In [ ]:
df['BROWSE_NODE_ID'].value_counts()

In [ ]:
## preprocess train data

In [13]:
df = pd.read_csv('dataset/train.csv',error_bad_lines=False,quoting=csv.QUOTE_NONE,escapechar = "\\")
df

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4
...,...,...,...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,Still want to use your favorite earphones/head...,"[Indian Connectors: Made for Indian sockets, t...",SHOPBELL,14790


In [20]:
df['TITLE'] = df['TITLE'].apply(formatname)

In [21]:
df['DESCRIPTION'] = df['DESCRIPTION'].apply(createDescription)

In [22]:
df['BULLET_POINTS'] = df['BULLET_POINTS'].apply(createDescription)

In [23]:
df['BRAND'] = df['BRAND'].apply(createBrand)

In [24]:
df.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,pete the cat bedtime blue doll inch,pete the cat is the coolest most popular cat i...,pete the cat bedtime blue plush dollbased on t...,MerryMakers,0
1,the new yorker refrigerator magnet x,the new yorker handsome cello wrapped hard mag...,cat in a tea cup by new yorker cover artist gu...,TheNewYorker,1
2,the ultimate selfsufficiency handbook a comple...,0,skyhorse publishing,imusti,2
3,amway nutrilite kid chewable iron tablet,0,nutrilite kidschewable iron tabletsquantity ta...,Amway,3
4,teacher planner company lesson academic teache...,0,0,0,4


In [27]:
Y = df['BROWSE_NODE_ID']
X = df.drop(columns = ['BROWSE_NODE_ID'])
X

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,pete the cat bedtime blue doll inch,pete the cat is the coolest most popular cat i...,pete the cat bedtime blue plush dollbased on t...,MerryMakers
1,the new yorker refrigerator magnet x,the new yorker handsome cello wrapped hard mag...,cat in a tea cup by new yorker cover artist gu...,TheNewYorker
2,the ultimate selfsufficiency handbook a comple...,0,skyhorse publishing,imusti
3,amway nutrilite kid chewable iron tablet,0,nutrilite kidschewable iron tabletsquantity ta...,Amway
4,teacher planner company lesson academic teache...,0,0,0
...,...,...,...,...
2903019,premium aviator sunglass hd polarized bright g...,these premium aviator sunglass with color opti...,frame size lens height lens width mm nosebridg...,Generic
2903020,social distance sticker set of sticker slip re...,set of print social distancing sticker self ad...,safety sticker set of to maintain crowd contro...,Generic
2903021,torrto face shield pack of with adjustable ela...,complete face protection torrto face shield ef...,micron pack of pcscomplete face protection tor...,TORR-TO
2903022,typec to mm for oppo pro typec to mm audio jac...,still want to use your favorite earphonesheadp...,indian connector made for indian socket the wa...,SHOPBELL
